In [18]:
import numpy as np
import math

In [44]:

car_positionsreal = {'A':[679857.2739, 419317.2236],
                 'B': [679858.8576, 419333.4677],
                 'C': [679859.7071, 419307.1674],
                 'D': [679859.6912, 419340.4366],
                 'E': [679860.8342, 419357.7712],}
# 假设已知的车辆位置信息 优化前
car_positions = {'A': [679857.06, 419317.1297],
                 'B': [679857.94, 419332.56],
                 'C': [679859.44, 419305.3], 
                 'D': [679859.56, 419339.47], 
                 'E': [679860.75, 419356.9],}

In [47]:
l = distance(car_positions['A'],car_positionsreal['A'])
print('迭代中的欧氏距离',l)   
# 原来是2.26 现变为1.88

迭代中的欧氏距离 0.2336031249692711


In [46]:
def length(K):
    # print(K)
    global car_positionsreal  # 将car_positionsreal声明为全局变量
    if K in car_positionsreal:
        m = distance(car_positionsreal[chr(ord(K)-1)], car_positionsreal[K])
        # print(m)
    else:
        print(f"Key '{B}' not found in car_positionsreal dictionary")
    return m
#车辆K到K-1车的距离

def distance(A, B):
    return np.linalg.norm(np.array(A) - np.array(B))
# 初始化最小距离和对应的A点位置
min_dist = np.inf
min_a_pos = None
def predict_position(pos, angle, dist):
    x = pos[0] + np.sin(math.radians(angle)) * dist
    y = pos[1] + np.cos(math.radians(angle)) * dist
    return x, y
# 定义y轴正方向为北方，x轴正方向为东。定义angle(A,B)为从A观察B时，从y轴正方向出发顺时针移动与AB形成的夹角
#这么做只是为了仿真，而正常场景中这些数据源于传感器



def angle(A, B):
    x = B[0] - A[0]
    y = B[1] - A[1]
    angle_rad = np.arctan2(y, x)
    angle_deg = np.rad2deg(angle_rad)
    return (90 - angle_deg) % 360


In [48]:

# 设置均值和方差
mu, sigma = 0, 0.1

# 生成高斯白噪声
noise = np.random.normal(mu, sigma, 1)

dB = distance(car_positionsreal['A'], car_positionsreal['B'])
print(dB)
dB += noise
print(dB)

16.321117930409383
[16.37397755]


In [49]:
print(angle(car_positionsreal['B'],car_positionsreal['A']))
a =angle(car_positionsreal['B'],car_positionsreal['A'])
a = a+ noise
print(a)
# angle(car_positionsreal['B'],car_positionsreal['A']) = angle(car_positionsreal['B'],car_positionsreal['A'])+noise

185.5683885551037
[185.62124817]


In [50]:
import math
# 对第一个点进行预测：

m1 = distance(car_positions['A'], car_positions['B'])#第一步预测出的A坐标和B做欧氏距离 
print('m1',m1)# m1 = 17.611998750871642
err = abs(m1-dB)
min_a_pos = car_positions['A']
print("初始err",err)
for car in car_positions:
    if car not in ['B','A']:
        print('car',car)
        print('err',err)

        # 计算基点与A车辆之间的距离和角度信息
        dist = distance(car_positionsreal['A'], car_positionsreal[car])
        print('dist',dist)
        print(angle(car_positionsreal[car],car_positionsreal['A']))
        # 计算预测的A点位置,用car观察A
        x1k, y1k = predict_position(car_positions[car], angle(car_positionsreal[car],car_positionsreal['A']), dist)
        PRE_A=[x1k, y1k]
        # 计算预测位置与B车辆之间的欧氏距离
        print('观察A点的位置：',PRE_A)
        m = distance(PRE_A, car_positions['B'])
        err1=abs(m-dB)
        print('err1:',err1)
        # 如果比之前的最小距离小，则更新最小距离和对应的A点位置
        if err1 < err:
            err = err1
            print("err1 < err后更新的err",err)
            min_a_pos = [x1k, y1k]
            print('min_a_pos',min_a_pos)
            car_positions['A'] = min_a_pos # 最终确定K车辆的位置
        # print('****')
        # car_positions['A'] = np.array(car_positions['A'])
        # car_positionsreal['A'] = np.array(car_positionsreal['A'])

print("car_positions['A']",car_positions['A'])
y = distance(car_positions['A'] ,car_positionsreal['A'])
print('迭代中的欧氏距离',y)

m1 15.455373113905322
初始err [0.91860443]
car C
err [0.91860443]
dist 10.346382009227272
346.3981205463924
观察A点的位置： [679857.0068, 419315.35620000004]
err1: [0.85511403]
err1 < err后更新的err [0.85511403]
min_a_pos [679857.0068, 419315.35620000004]
car D
err [0.85511403]
dist 23.33852412406031
185.9451029407146
观察A点的位置： [679857.1427000001, 419316.257]
err1: [0.05149317]
err1 < err后更新的err [0.05149317]
min_a_pos [679857.1427000001, 419316.257]
car E
err [0.05149317]
dist 40.703606742515305
185.01801178754494
观察A点的位置： [679857.1897, 419316.35240000003]
err1: [0.14901999]
car_positions['A'] [679857.1427000001, 419316.257]
迭代中的欧氏距离 0.9754634796185507


In [51]:
for K in car_positions:#通过car点预测K车
    if K != 'A':
        # 观察K点以外的点,遍历其他所有车辆作为基点
        for car in car_positions:
            if car not in ['K']:
                m1 = distance(car_positions[K], car_positionsreal[chr(ord(K)-1)])#car_positions[chr(ord(K)-1)]为已经处理过的点
                print('m1',m1)
                K = str(K)
                print('被观察车辆K',K)
                print('此时观察的出发点CAR',car)
                dK = length(K) #真是距离of 车K和车K-1
                # print(dK)
                err = abs(m1-dK)
                print('err',err)
                min_a_pos = car_positions[K]
                # # 计算基点与A车辆之间的距离和角度信息
                dist = distance(car_positionsreal[K], car_positionsreal[car])
                angle1 = angle(car_positionsreal[car], car_positionsreal[K])
                print('car_positions[car], car_positions[K]',car_positions[car], car_positions[K])
                print('dist',dist)
                print('angle',angle1)
                # # 计算预测的K点位置
                x1k, y1k = predict_position(car_positions[car], angle1, dist)
                PRE_K=[x1k, y1k]
                print('观察k点的位置：',PRE_K)
                print('k点真实位置',car_positionsreal[K])

                # # 计算预测位置与B车辆之间的欧氏距离
                m3 = distance(PRE_K, car_positionsreal[chr(ord(K)-1)])
                err1=abs(m3-dK)
                print('m3',m3)
                print('err1',err1)
                # # 如果比之前的最小距离小，则更新最小距离和对应的A点位置
                if err1 < err:
                    min_a_pos = [x1k, y1k]
                    car_positions[K] = min_a_pos # 最终确定K车辆的位置
                # print('****')
                # car_positions[K] = np.array(car_positions[K])
                # car_positionsreal[K] = np.array(car_positionsreal[K])
                # distance = np.linalg.norm(car_positions[K] - car_positionsreal[K])
                # print('迭代中的欧氏距离',distance)

    print("-----")
    print("哪辆车:",K)
    print('car_positions[K]改进',car_positions[K])
    print("-----")

# print('car_positions[A]',car_positions['A'])
for K in car_positions:
    # 将列表转换为NumPy数组
    print(K)
    car_positions[K] = np.array(car_positions[K])
    car_positionsreal[K] = np.array(car_positionsreal[K])
    distance = np.linalg.norm(car_positions[K] - car_positionsreal[K])
    print('此时的欧氏距离',distance)  # 输出欧几里得距离

-----
哪辆车: A
car_positions[K]改进 [679857.1427000001, 419316.257]
-----
m1 15.35085841798426
被观察车辆K B
此时观察的出发点CAR A
err 0.9702595124251232
car_positions[car], car_positions[K] [679857.1427000001, 419316.257] [679857.94, 419332.56]
dist 16.321117930409383
angle 5.568388555103681
观察k点的位置： [679858.7264, 419332.50109999994]
k点真实位置 [679858.8576, 419333.4677]
m3 15.346392491307723
err1 0.9747254391016593
m1 15.35085841798426
被观察车辆K B
此时观察的出发点CAR B
err 0.9702595124251232
car_positions[car], car_positions[K] [679857.94, 419332.56] [679857.94, 419332.56]
dist 0.0
angle 90.0
观察k点的位置： [679857.94, 419332.56]
k点真实位置 [679858.8576, 419333.4677]
m3 15.35085841798426
err1 0.9702595124251232
m1 15.35085841798426
被观察车辆K B
此时观察的出发点CAR C
err 0.9702595124251232
car_positions[car], car_positions[K] [679859.44, 419305.3] [679857.94, 419332.56]
dist 26.314015853536233
angle 358.14998883398704
观察k点的位置： [679858.5904999999, 419331.6003]
k点真实位置 [679858.8576, 419333.4677]
m3 14.436860408299333
err1 1.8842575221100493